In [1]:
import os
import json
import tensorflow as tf

import MeCab
import pandas as pd

from transformers import Wav2Vec2CTCTokenizer

os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [2]:
with open(r"E:\Datasets\ASR-dataset\vocab.json", "r") as f:
    vocab = json.load(f)

print(vocab)

{'<pad>': 0, '<s>': 1, '</s>': 2, '<unk>': 3, ' ': 4, 'a': 5, 'o': 6, 'i': 7, 'u': 8, 'n': 9, 'e': 10, 'k': 11, 't': 12, 'r': 13, 'm': 14, 's': 15, 'sh': 16, 'd': 17, 'g': 18, 'w': 19, 'y': 20, 'b': 21, 'h': 22, 'ts': 23, 'j': 24, 'ch': 25, 'z': 26, 'p': 27, 'f': 28, 'ky': 29, 'ry': 30, 'gy': 31, 'hy': 32, 'ny': 33, 'by': 34, 'my': 35, 'py': 36}


In [3]:
with open("D:\School-stuff\Sem-2\PR-Project\HoloASR\hira_vocab.json", "r", encoding="utf-8") as f:
    hira_vocab = json.load(f)

print(hira_vocab)

{'あ': 'a', 'い': 'i', 'う': 'u', 'え': 'e', 'お': 'o', 'か': 'ka', 'き': 'ki', 'く': 'ku', 'け': 'ke', 'こ': 'ko', 'さ': 'sa', 'し': 'shi', 'す': 'su', 'せ': 'se', 'そ': 'so', 'た': 'ta', 'ち': 'chi', 'つ': 'tsu', 'て': 'te', 'と': 'to', 'な': 'na', 'に': 'ni', 'ぬ': 'nu', 'ね': 'ne', 'の': 'no', 'は': 'ha', 'ふ': 'fu', 'へ': 'he', 'ほ': 'ho', 'ひ': 'hi', 'ま': 'ma', 'め': 'me', 'も': 'mo', 'み': 'mi', 'む': 'mu', 'や': 'ya', 'よ': 'yu', 'ゆ': 'yo', 'ら': 'ra', 'り': 'ri', 'る': 'ru', 'れ': 're', 'ろ': 'ro', 'わ': 'wa', 'を': 'wo', 'ん': 'n', 'が': 'ga', 'ぎ': 'gi', 'ぐ': 'gu', 'げ': 'ge', 'ご': 'go', 'ざ': 'za', 'ぜ': 'ze', 'ず': 'zu', 'じ': 'zi', 'ぞ': 'zo', 'だ': 'da', 'ど': 'do', 'づ': 'du', 'で': 'de', 'ぢ': 'di', 'ぼ': 'bo', 'び': 'bi', 'ぶ': 'bu', 'べ': 'be', 'ば': 'ba', 'ぱ': 'pa', 'ぴ': 'pi', 'ぷ': 'pu', 'ぺ': 'pe', 'ぽ': 'po', 'きゃ': 'kya', 'きゅ': 'kyu', 'きょ': 'kyo', 'ぎゃ': 'gya', 'ぎゅ': 'gyu', 'ぎょ': 'gyo', 'しゃ': 'sha', 'しゅ': 'shu', 'しょ': 'sho', 'じゃ': 'ja', 'じゅ': 'ju', 'じょ': 'jo', 'ちゃ': 'cha', 'ちゅ': 'chu', 'ちょ': 'cho', 'にゃ': 'nya', 'にゅ': 'nyu', 'にょ

In [19]:
special_tokens = ["<pad>", "<s>", "</s>", "<unk>", " "]

mora_vocab = {mora: i for i, mora in enumerate(special_tokens + list(hira_vocab.values()))}

with open("D:\School-stuff\Sem-2\PR-Project\HoloASR\mora2idx_vocab.json", "w") as f:
    json.dump(mora_vocab, f, indent=4)

In [20]:
mora_vocab = {i:mora for i, mora in enumerate(special_tokens + list(hira_vocab.keys()))}

with open("D:\School-stuff\Sem-2\PR-Project\HoloASR\idx2hira_vocab.json", "w", encoding="utf-8") as f:
    json.dump(mora_vocab, f, indent=4, ensure_ascii=True)

In [6]:
data = pd.read_csv("E:\Datasets\ASR-dataset\ASRDataset.csv")
data

,path,sentence,corpus,romaji,length
0,BASIC5000_0896.wav,ポル は 目 を 手 で 覆い ながら 立っ て い た,jsut,poru wa me wo te de ooinagara tatte ita,48000
1,BASIC5000_4182.wav,太い 尾 の 肉 が 特に 美味 と さ れる,jsut,futoi o no niku ga tokuni bimi to sareru,48000
2,nowaki-by-soseki-natsume-02203.wav,これ が さいだ と 引き合わ せる,kokoro,kore ga saida to hikiawaseru,48112
3,kusamakura-by-soseki-natsume-02837.wav,趣味 の 何物 たる を も 心得 ぬ,kokoro,shumi no nanimono taru wo mo kokoroenu,48112
4,eijitsu-syohin-by-soseki-natsume-00116.wav,けれど も それ が なかなか に 動か ない,kokoro,keredo mo sore ga nakanaka ni ugokanai,48112
...,...,...,...,...,...
4995,meian-by-soseki-natsume-10482.wav,同時 に それ より はるか 重大 な 主題 の ため に 悩まさ れ て い た,kokoro,douji ni sore yori haruka juudai na shudai no ...,79877
4996,eijitsu-syohin-by-soseki-natsume-00433.wav,自分 の 部屋 へ 這入っ て 寝 て しまう 母 は よほど 前 に なくなっ た,kokoro,jibun no heya e haitte nete shimau haha wa yoh...,79877
4997,kouyahijiri-by-kyoka-izumi-00617.wav,と もの と いいかけ て おやじ は 少年 の そば へ にじり寄っ て,kokoro,to mono to iikakete oyaji wa shounen no soba e...,79877
4998,BASIC5000_1367.wav,鹿児島 に は 一 度 も 行っ た こと が あり ませ ん が 今年 の 夏 行く つも...,jsut,kagoshima ni wa ichi do mo okonatta koto ga ar...,80000


In [21]:
# Tagger
wakati = MeCab.Tagger("-Owakati")

# idx2hira
with open("D:\School-stuff\Sem-2\PR-Project\HoloASR\idx2hira_vocab.json", "r") as f:
    idx2hira = json.load(f)

# mora2idx
tokenizer = Wav2Vec2CTCTokenizer(
    vocab_file="D:\School-stuff\Sem-2\PR-Project\HoloASR\mora2idx_vocab.json",
    word_delimiter_token=" ",
    do_lower_case=False,
    )
tokenizer

PreTrainedTokenizer(name_or_path='', vocab_size=110, model_max_len=1000000000000000019884624838656, is_fast=False, padding_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>'})